# Getting Started with Authority Nanos

Welcome to Authority Nanos! This notebook provides an interactive introduction to the Authority Kernel Python SDK.

## What is Authority Nanos?

Authority Nanos is a **capability-based security kernel** designed for AI agent workloads. It provides:

- **Typed Heap**: Secure memory management with JSON Patch updates
- **Authorization**: Fine-grained capability-based access control
- **Tool Execution**: Sandboxed WASM tool execution
- **LLM Inference**: Policy-controlled LLM access
- **Audit Logging**: Complete audit trail of all operations

The kernel enforces security policies that control what operations AI agents can perform, ensuring safe and auditable AI systems.

## Simulation Mode

This notebook uses **simulation mode** by default, which means:
- No actual kernel binary is required
- All operations run in-memory in Python
- Perfect for learning, testing, and development

Let's get started!

## 1. Installation

First, install the Authority Nanos SDK:

In [ ]:
# Install the SDK (uncomment if not already installed)
# %pip install authority-nanos

## 2. Importing the SDK

Import the main components we'll use:

In [ ]:
import json
from authority_nanos import AuthorityKernel, AuthorityKernelError

print("Authority Nanos SDK imported successfully!")

## 3. Hello World Example

Let's start with the simplest possible example - allocating an object in the typed heap and reading it back.

The `AuthorityKernel` class is the main entry point. We use `simulate=True` to run without a real kernel:

In [ ]:
# Connect to Authority Kernel in simulation mode
with AuthorityKernel(simulate=True) as ak:
    # Create a simple greeting object
    data = json.dumps({"message": "Hello, World!"}).encode()
    
    # Allocate it in the typed heap
    handle = ak.alloc("greeting", data)
    print(f"Allocated object with handle: {handle}")
    
    # Read it back
    result = ak.read(handle)
    obj = json.loads(result.decode())
    
    print(f"Read back: {obj['message']}")

### What just happened?

1. We created an `AuthorityKernel` context with `simulate=True`
2. We serialized a Python dict to JSON bytes
3. We allocated it in the typed heap with type `"greeting"`
4. The kernel returned a **Handle** - a unique reference to the object
5. We read the object back using the handle

The handle contains:
- `id`: A unique identifier for the object
- `version`: The current version (used for concurrent updates)

## 4. Checking Simulation Mode

You can verify whether you're running in simulation mode:

In [ ]:
with AuthorityKernel(simulate=True) as ak:
    if ak.is_simulated():
        print("Running in SIMULATION mode")
        print("  - No kernel binary required")
        print("  - All operations run in-memory")
        print("  - Perfect for development and testing")
    else:
        print("Running with REAL kernel")

## 5. Basic Heap Operations

The typed heap is the core data storage mechanism in Authority Kernel. It supports four operations:

| Operation | Description |
|-----------|-------------|
| `alloc` | Allocate a new object |
| `read` | Read an object's current value |
| `write` | Update an object using JSON Patch |
| `delete` | Soft-delete an object |

Let's explore each one:

### 5.1 Allocation (`alloc`)

Create a new object in the typed heap:

In [ ]:
with AuthorityKernel(simulate=True) as ak:
    # Allocate a counter object
    counter_data = json.dumps({
        "value": 0,
        "name": "my_counter",
        "created": "2024-01-01"
    }).encode()
    
    handle = ak.alloc("counter", counter_data)
    
    print(f"Allocated object:")
    print(f"  Type: counter")
    print(f"  Handle ID: {handle.id}")
    print(f"  Version: {handle.version}")

### 5.2 Reading (`read`)

Retrieve an object's current value:

In [ ]:
with AuthorityKernel(simulate=True) as ak:
    # Create an object
    user_data = json.dumps({
        "username": "alice",
        "email": "alice@example.com",
        "role": "admin"
    }).encode()
    
    handle = ak.alloc("user", user_data)
    
    # Read it back
    data = ak.read(handle)
    user = json.loads(data.decode())
    
    print("Read user object:")
    print(json.dumps(user, indent=2))

### 5.3 Updating (`write`)

Update an object using **JSON Patch** (RFC 6902). This provides atomic, fine-grained updates:

Common patch operations:
- `{"op": "replace", "path": "/field", "value": newValue}` - Replace a value
- `{"op": "add", "path": "/field", "value": newValue}` - Add a new field
- `{"op": "remove", "path": "/field"}` - Remove a field

In [ ]:
with AuthorityKernel(simulate=True) as ak:
    # Create a counter
    counter_data = json.dumps({"value": 0}).encode()
    handle = ak.alloc("counter", counter_data)
    
    print("Initial state:")
    data = ak.read(handle)
    print(f"  {json.loads(data.decode())}")
    print(f"  Version: {handle.version}")
    
    # Update using JSON Patch
    patch = json.dumps([
        {"op": "replace", "path": "/value", "value": 42}
    ]).encode()
    
    new_version = ak.write(handle, patch)
    
    print("\nAfter update:")
    data = ak.read(handle)
    print(f"  {json.loads(data.decode())}")
    print(f"  New version: {new_version}")

### Multiple Updates in One Patch

You can apply multiple changes atomically:

In [ ]:
with AuthorityKernel(simulate=True) as ak:
    # Create an object with multiple fields
    data = json.dumps({
        "name": "Task 1",
        "status": "pending",
        "priority": 1
    }).encode()
    
    handle = ak.alloc("task", data)
    
    print("Before:")
    print(json.dumps(json.loads(ak.read(handle).decode()), indent=2))
    
    # Apply multiple changes at once
    patch = json.dumps([
        {"op": "replace", "path": "/status", "value": "completed"},
        {"op": "add", "path": "/completed_at", "value": "2024-01-15"},
        {"op": "replace", "path": "/priority", "value": 0}
    ]).encode()
    
    ak.write(handle, patch)
    
    print("\nAfter:")
    print(json.dumps(json.loads(ak.read(handle).decode()), indent=2))

### 5.4 Deleting (`delete`)

Soft-delete an object (marks it as deleted but preserves data for audit):

In [ ]:
with AuthorityKernel(simulate=True) as ak:
    # Create an object
    data = json.dumps({"temp": "data"}).encode()
    handle = ak.alloc("temporary", data)
    
    print(f"Created object with handle ID: {handle.id}")
    
    # Read it
    result = ak.read(handle)
    print(f"Read: {json.loads(result.decode())}")
    
    # Delete it
    ak.delete(handle)
    print("Object deleted")
    
    # Trying to read a deleted object will fail
    try:
        ak.read(handle)
    except Exception as e:
        print(f"Cannot read deleted object: {type(e).__name__}")

## 6. Complete Example: Task Manager

Let's put it all together with a simple task manager:

In [ ]:
with AuthorityKernel(simulate=True) as ak:
    print("=== Simple Task Manager ===")
    print()
    
    # Create a task list
    task_list = json.dumps({
        "name": "My Tasks",
        "tasks": []
    }).encode()
    
    list_handle = ak.alloc("task_list", task_list)
    print(f"Created task list (handle ID: {list_handle.id})")
    
    # Add some tasks
    tasks_to_add = [
        {"id": 1, "title": "Learn Authority Nanos", "done": False},
        {"id": 2, "title": "Build an AI agent", "done": False},
        {"id": 3, "title": "Deploy to production", "done": False}
    ]
    
    for task in tasks_to_add:
        patch = json.dumps([
            {"op": "add", "path": "/tasks/-", "value": task}
        ]).encode()
        ak.write(list_handle, patch)
        print(f"  Added task: {task['title']}")
    
    print()
    
    # Read current state
    current = json.loads(ak.read(list_handle).decode())
    print("Current tasks:")
    for task in current["tasks"]:
        status = "[x]" if task["done"] else "[ ]"
        print(f"  {status} {task['title']}")
    
    print()
    
    # Mark first task as done
    patch = json.dumps([
        {"op": "replace", "path": "/tasks/0/done", "value": True}
    ]).encode()
    ak.write(list_handle, patch)
    print("Marked 'Learn Authority Nanos' as done!")
    
    print()
    
    # Final state
    final = json.loads(ak.read(list_handle).decode())
    print("Updated tasks:")
    for task in final["tasks"]:
        status = "[x]" if task["done"] else "[ ]"
        print(f"  {status} {task['title']}")

## Summary

In this notebook, you learned:

1. **What Authority Nanos is** - A capability-based security kernel for AI agents
2. **Simulation mode** - Test without a real kernel using `simulate=True`
3. **Basic heap operations**:
   - `alloc(type, data)` - Create objects
   - `read(handle)` - Retrieve objects
   - `write(handle, patch)` - Update objects with JSON Patch
   - `delete(handle)` - Remove objects

## Next Steps

Continue with the next notebooks:
- **02_authorization.ipynb** - Learn about capability-based authorization
- **03_building_agents.ipynb** - Build AI agents with Authority Nanos
- **04_langchain_integration.ipynb** - Integrate with LangChain